In [ ]:
from fastai.vision.all import *
from skimage import measure
from skimage.transform import rescale, resize
from skimage.util import crop, montage
from skimage.morphology import label, square, dilation, watershed
from skimage.io import imsave


from tqdm.notebook import trange, tqdm
from PIL import Image

In [ ]:

df= pd.read_csv('../input/cropped-imges-and-masks/crops_with_ships.csv')


##### len(df)

### Tratamiento del CSV
Como podemos observar, en el CSV va a aparecer una entrada por barco, por lo tanto, cuando aparece más de un barco en la misma imagen, ésta aparecerá tantas veces en el csv como barcos contenga. Vamos a agrupar todos los barcos de la imagen en una sola entrada, agrupando por ImageId y uniendo encoded pixels con un espacio de separación. Además de ello, para mayor facilidad en el entrenamiento posterior añadiremos un nuevo campo que llamaremos "has_ship" que vale 1 en caso de contener barcos y 0 en caso de no contenerlos. https://blog.softhints.com/python-detect-prevent-typeerror/

In [ ]:
def image_open(img_path):
    return np.array(Image.open(img_path))

def apply_mask(image,mask):
    imax,jmax=mask.shape
    image_masked=np.copy(image)
    for i in range(imax):
        for j in range(jmax):
            if mask[i,j]==1:
                image_masked[i,j,[0,0]]=170
    return image_masked


### Clasificación
Empezar entrenando un modelo que clasifique las imágenes de entrada etiquetándolas como imágenes que contenen barcos e imagenes que no contienen barcos. Debido a que el dataset contienen muchás más imágenes que no contienen barcos, primero utilizarémos este modelo para **eliminar todas las imágnes que no contengan barcos posibles** para, posteriormente, utilizar un segundo modelo **entrenado exclusivemente para la segmentación en imagénes que contengan barcos**. 


In [ ]:
def get_x(r): return os.path.join('../input/cropped-imges-and-masks/croppedmasks/crops',r['img_name'])
def get_y(r): return r['has_ships']



In [ ]:
dblock= DataBlock(blocks=(ImageBlock,CategoryBlock),get_x=get_x, get_y=get_y)
dls=dblock.dataloaders(df, bs=64)


In [ ]:
dls.train.show_batch(max_n=4,unique=True)

In [ ]:
dls.valid.show_batch(max_n=4)

In [ ]:
dls.show_batch(nrows=2,ncols=4)

In [ ]:
learn=cnn_learner(dls,resnet34, metrics=accuracy,loss_func=CrossEntropyLossFlat())



In [ ]:
learn.lr_find()

In [ ]:
learn.fine_tune(15, base_lr=2e-3,freeze_epochs=1)

In [ ]:
learn.save('Resnet34_256_crops')


In [ ]:
#deberías hacer esto para cargar los pesos aprendidos
#learn.load('Resnet34')

In [ ]:
learn.show_results()

In [ ]:
learn.recorder.plot_loss()